In [0]:
bronze_output = dbutils.jobs.taskValues.get(taskKey="Bronze", key="bronze_output")
silver_data = dbutils.jobs.taskValues.get(taskKey="Silver", key="silver_output")

# Access individual variables
start_date = bronze_output.get("start_date", "")
silver_adls = bronze_output.get("silver_adls", "")
gold_adls = bronze_output.get("gold_adls", "")

silver_output_path = f"{silver_adls}earthquake_events_silver/"

print(f"Start Date: {start_date}, Gold ADLS: {gold_adls}")

In [0]:
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType
# Ensure the below library is installed on your cluster
import reverse_geocoder as rg
from datetime import date, timedelta

In [0]:
df = spark.read.parquet(silver_data).filter(col('time') > start_date)

In [0]:
df_with_sig_class = \
                            df.\
                                withColumn('sig_class', 
                                            when(col("sig") < 100, "Low").\
                                            when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                            otherwise("High")
                                            )

In [0]:
df_with_location_sig_class.head()
gold_output_path = f"{gold_adls}earthquake_events_gold/"
df_with_sig_class.write.mode('append').parquet(gold_output_path)